In [1]:
%matplotlib inline

In [2]:
from galsize_models.measurements import load_umachine_sdss_with_meert15
umachine_sdss, is_complete = load_umachine_sdss_with_meert15()

good_profile_mask = ~np.isnan(umachine_sdss['r50_magr_kpc_meert15'])
good_profile_mask *= ~np.isnan(umachine_sdss['logMB_mendel13'])
good_profile_mask *= ~np.isnan(umachine_sdss['logMD_mendel13'])
good_profile_mask *= umachine_sdss['logMD_mendel13'] > 0
good_profile_mask *= umachine_sdss['logMB_mendel13'] > 0

mask = is_complete & good_profile_mask
sdss = umachine_sdss[mask]
print("Number of M*-complete SDSS galaxies with good profile measurements = {0}".format(len(sdss)))


Number of M*-complete SDSS galaxies with good profile measurements = 223253


In [3]:
from galsize_models.measurements import mendel13_bulge_to_total

sf_mask = sdss['ssfr'] >= -10.75
q_mask = sdss['ssfr'] < -11.25
gv_mask = ~sf_mask & ~q_mask

sm10p0_mask = sdss['sm'] > 10
sm10p5_mask = sdss['sm'] > 10.5

meert15_bulge_dom_mask = sdss['bulge_to_total_rband_meert15'] > 0.75
meert15_disk_dom_mask = sdss['bulge_to_total_rband_meert15'] < 0.25

mendel13_bulge_dom_mask = mendel13_bulge_to_total(sdss) > 0.75
mendel13_disk_dom_mask = mendel13_bulge_to_total(sdss) < 0.25

In [4]:
sdss_sm10p0 = sdss[sm10p0_mask]
sdss_sm10p5 = sdss[sm10p5_mask]

sdss_sm10p0_sf = sdss[sm10p0_mask & sf_mask]
sdss_sm10p5_sf = sdss[sm10p5_mask & sf_mask]

sdss_sm10p0_gv = sdss[sm10p0_mask & gv_mask]
sdss_sm10p5_gv = sdss[sm10p5_mask & gv_mask]

sdss_sm10p0_q = sdss[sm10p0_mask & q_mask]
sdss_sm10p5_q = sdss[sm10p5_mask & q_mask]

sdss_sm10p0_meert15_bulge_dom = sdss[sm10p0_mask & meert15_bulge_dom_mask]
sdss_sm10p5_meert15_bulge_dom = sdss[sm10p5_mask & meert15_bulge_dom_mask]

sdss_sm10p0_mendel13_bulge_dom = sdss[sm10p0_mask & mendel13_bulge_dom_mask]
sdss_sm10p5_mendel13_bulge_dom = sdss[sm10p5_mask & mendel13_bulge_dom_mask]

sdss_sm10p0_meert15_disk_dom = sdss[sm10p0_mask & meert15_disk_dom_mask]
sdss_sm10p5_meert15_disk_dom = sdss[sm10p5_mask & meert15_disk_dom_mask]

sdss_sm10p0_mendel13_disk_dom = sdss[sm10p0_mask & mendel13_disk_dom_mask]
sdss_sm10p5_mendel13_disk_dom = sdss[sm10p5_mask & mendel13_disk_dom_mask]

sdss_sm10p0_sf_meert15_bulge_dom = sdss[sm10p0_mask & sf_mask & meert15_bulge_dom_mask]
sdss_sm10p5_sf_meert15_bulge_dom = sdss[sm10p5_mask & sf_mask & meert15_bulge_dom_mask]

sdss_sm10p0_gv_meert15_bulge_dom = sdss[sm10p0_mask & gv_mask & meert15_bulge_dom_mask]
sdss_sm10p5_gv_meert15_bulge_dom = sdss[sm10p5_mask & gv_mask & meert15_bulge_dom_mask]

sdss_sm10p0_q_meert15_bulge_dom = sdss[sm10p0_mask & q_mask & meert15_bulge_dom_mask]
sdss_sm10p5_q_meert15_bulge_dom = sdss[sm10p5_mask & q_mask & meert15_bulge_dom_mask]

sdss_sm10p0_sf_mendel13_bulge_dom = sdss[sm10p0_mask & sf_mask & mendel13_bulge_dom_mask]
sdss_sm10p5_sf_mendel13_bulge_dom = sdss[sm10p5_mask & sf_mask & mendel13_bulge_dom_mask]

sdss_sm10p0_gv_mendel13_bulge_dom = sdss[sm10p0_mask & gv_mask & mendel13_bulge_dom_mask]
sdss_sm10p5_gv_mendel13_bulge_dom = sdss[sm10p5_mask & gv_mask & mendel13_bulge_dom_mask]

sdss_sm10p0_q_mendel13_bulge_dom = sdss[sm10p0_mask & q_mask & mendel13_bulge_dom_mask]
sdss_sm10p5_q_mendel13_bulge_dom = sdss[sm10p5_mask & q_mask & mendel13_bulge_dom_mask]

sdss_sm10p0_sf_meert15_disk_dom = sdss[sm10p0_mask & sf_mask & meert15_disk_dom_mask]
sdss_sm10p5_sf_meert15_disk_dom = sdss[sm10p5_mask & sf_mask & meert15_disk_dom_mask]

sdss_sm10p0_gv_meert15_disk_dom = sdss[sm10p0_mask & gv_mask & meert15_disk_dom_mask]
sdss_sm10p5_gv_meert15_disk_dom = sdss[sm10p5_mask & gv_mask & meert15_disk_dom_mask]

sdss_sm10p0_q_meert15_disk_dom = sdss[sm10p0_mask & q_mask & meert15_disk_dom_mask]
sdss_sm10p5_q_meert15_disk_dom = sdss[sm10p5_mask & q_mask & meert15_disk_dom_mask]

sdss_sm10p0_sf_mendel13_disk_dom = sdss[sm10p0_mask & sf_mask & mendel13_disk_dom_mask]
sdss_sm10p5_sf_mendel13_disk_dom = sdss[sm10p5_mask & sf_mask & mendel13_disk_dom_mask]

sdss_sm10p0_gv_mendel13_disk_dom = sdss[sm10p0_mask & gv_mask & mendel13_disk_dom_mask]
sdss_sm10p5_gv_mendel13_disk_dom = sdss[sm10p5_mask & gv_mask & mendel13_disk_dom_mask]

sdss_sm10p0_q_mendel13_disk_dom = sdss[sm10p0_mask & q_mask & mendel13_disk_dom_mask]
sdss_sm10p5_q_mendel13_disk_dom = sdss[sm10p5_mask & q_mask & mendel13_disk_dom_mask]

In [5]:
from galsize_models.measurements.split_sdss_sample_rhalf import sdss_sample_below_median_size

_small_size_mask = sdss_sample_below_median_size(sdss_sm10p0)
sdss_sm10p0_small = sdss_sm10p0[_small_size_mask]
sdss_sm10p0_large = sdss_sm10p0[~_small_size_mask]

_small_size_mask = sdss_sample_below_median_size(sdss_sm10p5)
sdss_sm10p5_small = sdss_sm10p5[_small_size_mask]
sdss_sm10p5_large = sdss_sm10p5[~_small_size_mask]

_small_size_mask = sdss_sample_below_median_size(sdss_sm10p0_sf)
sdss_sm10p0_sf_small = sdss_sm10p0_sf[_small_size_mask]
sdss_sm10p0_sf_large = sdss_sm10p0_sf[~_small_size_mask]

_small_size_mask = sdss_sample_below_median_size(sdss_sm10p5_sf)
sdss_sm10p5_sf_small = sdss_sm10p5_sf[_small_size_mask]
sdss_sm10p5_sf_large = sdss_sm10p5_sf[~_small_size_mask]

_small_size_mask = sdss_sample_below_median_size(sdss_sm10p0_gv)
sdss_sm10p0_gv_small = sdss_sm10p0_gv[_small_size_mask]
sdss_sm10p0_gv_large = sdss_sm10p0_gv[~_small_size_mask]

_small_size_mask = sdss_sample_below_median_size(sdss_sm10p5_gv)
sdss_sm10p5_gv_small = sdss_sm10p5_gv[_small_size_mask]
sdss_sm10p5_gv_large = sdss_sm10p5_gv[~_small_size_mask]

_small_size_mask = sdss_sample_below_median_size(sdss_sm10p0_q)
sdss_sm10p0_q_small = sdss_sm10p0_q[_small_size_mask]
sdss_sm10p0_q_large = sdss_sm10p0_q[~_small_size_mask]

_small_size_mask = sdss_sample_below_median_size(sdss_sm10p5_q)
sdss_sm10p5_q_small = sdss_sm10p5_q[_small_size_mask]
sdss_sm10p5_q_large = sdss_sm10p5_q[~_small_size_mask]

_small_size_mask = sdss_sample_below_median_size(sdss_sm10p0_meert15_disk_dom)
sdss_sm10p0_meert15_disk_dom_small = sdss_sm10p0_meert15_disk_dom[_small_size_mask]
sdss_sm10p0_meert15_disk_dom_large = sdss_sm10p0_meert15_disk_dom[~_small_size_mask]

_small_size_mask = sdss_sample_below_median_size(sdss_sm10p0_meert15_bulge_dom)
sdss_sm10p0_meert15_bulge_dom_small = sdss_sm10p0_meert15_bulge_dom[_small_size_mask]
sdss_sm10p0_meert15_bulge_dom_large = sdss_sm10p0_meert15_bulge_dom[~_small_size_mask]

_small_size_mask = sdss_sample_below_median_size(sdss_sm10p5_meert15_disk_dom)
sdss_sm10p5_meert15_disk_dom_small = sdss_sm10p5_meert15_disk_dom[_small_size_mask]
sdss_sm10p5_meert15_disk_dom_large = sdss_sm10p5_meert15_disk_dom[~_small_size_mask]

_small_size_mask = sdss_sample_below_median_size(sdss_sm10p5_meert15_bulge_dom)
sdss_sm10p5_meert15_bulge_dom_small = sdss_sm10p5_meert15_bulge_dom[_small_size_mask]
sdss_sm10p5_meert15_bulge_dom_large = sdss_sm10p5_meert15_bulge_dom[~_small_size_mask]


In [6]:
from galsize_models.measurements.measure_sdss_clustering import save_wp_measurement

import os
repo_dirname = "/Users/aphearin/work/repositories/python/galsize_models"
output_subdirname = "galsize_models/measurements/data/two_point_functions"
output_dirname = os.path.join(repo_dirname, output_subdirname)

sm_high = 12


In [7]:
save_wp_measurement(sdss_sm10p0, output_dirname, 'sdss_sm10p0', 10., sm_high)
save_wp_measurement(sdss_sm10p5, output_dirname, 'sdss_sm10p5', 10.5, sm_high)
print("\a")

Measuring wp for sdss_sm10p0 sample of 180459 galaxies
Measuring wp for sdss_sm10p5 sample of 138231 galaxies



In [8]:
save_wp_measurement(sdss_sm10p0_sf, output_dirname, 'sdss_sm10p0_sf', 10., sm_high)
save_wp_measurement(sdss_sm10p0_gv, output_dirname, 'sdss_sm10p0_gv', 10., sm_high)
save_wp_measurement(sdss_sm10p0_q, output_dirname, 'sdss_sm10p0_q', 10., sm_high)

save_wp_measurement(sdss_sm10p5_sf, output_dirname, 'sdss_sm10p5_sf', 10.5, sm_high)
save_wp_measurement(sdss_sm10p5_gv, output_dirname, 'sdss_sm10p5_gv', 10.5, sm_high)
save_wp_measurement(sdss_sm10p5_q, output_dirname, 'sdss_sm10p5_q', 10.5, sm_high)
print("\a")

Measuring wp for sdss_sm10p0_sf sample of 44861 galaxies
Measuring wp for sdss_sm10p0_gv sample of 18922 galaxies
Measuring wp for sdss_sm10p0_q sample of 116676 galaxies
Measuring wp for sdss_sm10p5_sf sample of 25237 galaxies
Measuring wp for sdss_sm10p5_gv sample of 14227 galaxies
Measuring wp for sdss_sm10p5_q sample of 98767 galaxies



In [9]:
save_wp_measurement(sdss_sm10p0_meert15_bulge_dom, output_dirname, 'sdss_sm10p0_meert15_bulge_dom', 10., sm_high)
save_wp_measurement(sdss_sm10p0_meert15_disk_dom, output_dirname, 'sdss_sm10p0_meert15_disk_dom', 10., sm_high)
save_wp_measurement(sdss_sm10p5_meert15_bulge_dom, output_dirname, 'sdss_sm10p5_meert15_bulge_dom', 10.5, sm_high)
save_wp_measurement(sdss_sm10p5_meert15_disk_dom, output_dirname, 'sdss_sm10p5_meert15_disk_dom', 10.5, sm_high)
print("\a")

Measuring wp for sdss_sm10p0_meert15_bulge_dom sample of 28547 galaxies
Measuring wp for sdss_sm10p0_meert15_disk_dom sample of 45693 galaxies
Measuring wp for sdss_sm10p5_meert15_bulge_dom sample of 24349 galaxies
Measuring wp for sdss_sm10p5_meert15_disk_dom sample of 28588 galaxies



In [10]:
save_wp_measurement(sdss_sm10p0_mendel13_bulge_dom, output_dirname, 'sdss_sm10p0_mendel13_bulge_dom', 10., sm_high)
save_wp_measurement(sdss_sm10p0_mendel13_disk_dom, output_dirname, 'sdss_sm10p0_mendel13_disk_dom', 10., sm_high)
save_wp_measurement(sdss_sm10p5_mendel13_bulge_dom, output_dirname, 'sdss_sm10p5_mendel13_bulge_dom', 10.5, sm_high)
save_wp_measurement(sdss_sm10p5_mendel13_disk_dom, output_dirname, 'sdss_sm10p5_mendel13_disk_dom', 10.5, sm_high)
print("\a")

Measuring wp for sdss_sm10p0_mendel13_bulge_dom sample of 64486 galaxies
Measuring wp for sdss_sm10p0_mendel13_disk_dom sample of 12704 galaxies
Measuring wp for sdss_sm10p5_mendel13_bulge_dom sample of 52886 galaxies
Measuring wp for sdss_sm10p5_mendel13_disk_dom sample of 5780 galaxies



In [11]:
save_wp_measurement(sdss_sm10p0_sf_meert15_bulge_dom, output_dirname, 'sdss_sm10p0_sf_meert15_bulge_dom', 10., sm_high)
save_wp_measurement(sdss_sm10p0_sf_meert15_disk_dom, output_dirname, 'sdss_sm10p0_sf_meert15_disk_dom', 10., sm_high)
save_wp_measurement(sdss_sm10p5_sf_meert15_bulge_dom, output_dirname, 'sdss_sm10p5_sf_meert15_bulge_dom', 10.5, sm_high)
save_wp_measurement(sdss_sm10p5_sf_meert15_disk_dom, output_dirname, 'sdss_sm10p5_sf_meert15_disk_dom', 10.5, sm_high)
print("\a")

Measuring wp for sdss_sm10p0_sf_meert15_bulge_dom sample of 2264 galaxies
Measuring wp for sdss_sm10p0_sf_meert15_disk_dom sample of 26364 galaxies
Measuring wp for sdss_sm10p5_sf_meert15_bulge_dom sample of 1574 galaxies
Measuring wp for sdss_sm10p5_sf_meert15_disk_dom sample of 13291 galaxies



In [12]:
save_wp_measurement(sdss_sm10p0_sf_mendel13_bulge_dom, output_dirname, 'sdss_sm10p0_sf_mendel13_bulge_dom', 10., sm_high)
save_wp_measurement(sdss_sm10p0_sf_mendel13_disk_dom, output_dirname, 'sdss_sm10p0_sf_mendel13_disk_dom', 10., sm_high)
save_wp_measurement(sdss_sm10p5_sf_mendel13_bulge_dom, output_dirname, 'sdss_sm10p5_sf_mendel13_bulge_dom', 10.5, sm_high)
save_wp_measurement(sdss_sm10p5_sf_mendel13_disk_dom, output_dirname, 'sdss_sm10p5_sf_mendel13_disk_dom', 10.5, sm_high)
print("\a")

Measuring wp for sdss_sm10p0_sf_mendel13_bulge_dom sample of 11442 galaxies
Measuring wp for sdss_sm10p0_sf_mendel13_disk_dom sample of 8454 galaxies
Measuring wp for sdss_sm10p5_sf_mendel13_bulge_dom sample of 7943 galaxies
Measuring wp for sdss_sm10p5_sf_mendel13_disk_dom sample of 2913 galaxies



In [13]:
save_wp_measurement(sdss_sm10p0_gv_meert15_bulge_dom, output_dirname, 'sdss_sm10p0_gv_meert15_bulge_dom', 10., sm_high)
save_wp_measurement(sdss_sm10p0_gv_meert15_disk_dom, output_dirname, 'sdss_sm10p0_gv_meert15_disk_dom', 10., sm_high)
save_wp_measurement(sdss_sm10p5_gv_meert15_bulge_dom, output_dirname, 'sdss_sm10p5_gv_meert15_bulge_dom', 10.5, sm_high)
save_wp_measurement(sdss_sm10p5_gv_meert15_disk_dom, output_dirname, 'sdss_sm10p5_gv_meert15_disk_dom', 10.5, sm_high)

save_wp_measurement(sdss_sm10p0_gv_mendel13_bulge_dom, output_dirname, 'sdss_sm10p0_gv_mendel13_bulge_dom', 10., sm_high)
save_wp_measurement(sdss_sm10p0_gv_mendel13_disk_dom, output_dirname, 'sdss_sm10p0_gv_mendel13_disk_dom', 10., sm_high)
save_wp_measurement(sdss_sm10p5_gv_mendel13_bulge_dom, output_dirname, 'sdss_sm10p5_gv_mendel13_bulge_dom', 10.5, sm_high)
save_wp_measurement(sdss_sm10p5_gv_mendel13_disk_dom, output_dirname, 'sdss_sm10p5_gv_mendel13_disk_dom', 10.5, sm_high)
print("\a")

Measuring wp for sdss_sm10p0_gv_meert15_bulge_dom sample of 1576 galaxies
Measuring wp for sdss_sm10p0_gv_meert15_disk_dom sample of 7178 galaxies
Measuring wp for sdss_sm10p5_gv_meert15_bulge_dom sample of 1235 galaxies
Measuring wp for sdss_sm10p5_gv_meert15_disk_dom sample of 5266 galaxies
Measuring wp for sdss_sm10p0_gv_mendel13_bulge_dom sample of 5391 galaxies
Measuring wp for sdss_sm10p0_gv_mendel13_disk_dom sample of 1622 galaxies
Measuring wp for sdss_sm10p5_gv_mendel13_bulge_dom sample of 4345 galaxies
Measuring wp for sdss_sm10p5_gv_mendel13_disk_dom sample of 913 galaxies



In [21]:
save_wp_measurement(sdss_sm10p0_q_meert15_bulge_dom, output_dirname, 'sdss_sm10p0_q_meert15_bulge_dom', 10., sm_high)
save_wp_measurement(sdss_sm10p0_q_meert15_disk_dom, output_dirname, 'sdss_sm10p0_q_meert15_disk_dom', 10., sm_high)
save_wp_measurement(sdss_sm10p5_q_meert15_bulge_dom, output_dirname, 'sdss_sm10p5_q_meert15_bulge_dom', 10.5, sm_high)
save_wp_measurement(sdss_sm10p5_q_meert15_disk_dom, output_dirname, 'sdss_sm10p5_q_meert15_disk_dom', 10.5, sm_high)

save_wp_measurement(sdss_sm10p0_q_mendel13_bulge_dom, output_dirname, 'sdss_sm10p0_q_mendel13_bulge_dom', 10., sm_high)
save_wp_measurement(sdss_sm10p0_q_mendel13_disk_dom, output_dirname, 'sdss_sm10p0_q_mendel13_disk_dom', 10., sm_high)
save_wp_measurement(sdss_sm10p5_q_mendel13_bulge_dom, output_dirname, 'sdss_sm10p5_q_mendel13_bulge_dom', 10.5, sm_high)
save_wp_measurement(sdss_sm10p5_q_mendel13_disk_dom, output_dirname, 'sdss_sm10p5_q_mendel13_disk_dom', 10.5, sm_high)
print("\a")

Measuring wp for sdss_sm10p0_q_meert15_bulge_dom sample of 24707 galaxies
Measuring wp for sdss_sm10p0_q_meert15_disk_dom sample of 12151 galaxies
Measuring wp for sdss_sm10p5_q_meert15_bulge_dom sample of 21540 galaxies
Measuring wp for sdss_sm10p5_q_meert15_disk_dom sample of 10031 galaxies
Measuring wp for sdss_sm10p0_q_mendel13_bulge_dom sample of 47653 galaxies
Measuring wp for sdss_sm10p0_q_mendel13_disk_dom sample of 2628 galaxies
Measuring wp for sdss_sm10p5_q_mendel13_bulge_dom sample of 40598 galaxies
Measuring wp for sdss_sm10p5_q_mendel13_disk_dom sample of 1954 galaxies



## Now for size-split clustering measurements

In [15]:
save_wp_measurement(sdss_sm10p0_small, output_dirname, 'sdss_sm10p0_small', 10., sm_high)
save_wp_measurement(sdss_sm10p0_large, output_dirname, 'sdss_sm10p0_large', 10., sm_high)
save_wp_measurement(sdss_sm10p5_small, output_dirname, 'sdss_sm10p5_small', 10.5, sm_high)
save_wp_measurement(sdss_sm10p5_large, output_dirname, 'sdss_sm10p5_large', 10.5, sm_high)


Measuring wp for sdss_sm10p0_small sample of 91943 galaxies
Measuring wp for sdss_sm10p0_large sample of 88516 galaxies
Measuring wp for sdss_sm10p5_small sample of 70154 galaxies
Measuring wp for sdss_sm10p5_large sample of 68077 galaxies


In [17]:
save_wp_measurement(sdss_sm10p0_sf_small, output_dirname, 'sdss_sm10p0_sf_small', 10., sm_high)
save_wp_measurement(sdss_sm10p0_sf_large, output_dirname, 'sdss_sm10p0_sf_large', 10., sm_high)
save_wp_measurement(sdss_sm10p0_gv_small, output_dirname, 'sdss_sm10p0_gv_small', 10., sm_high)
save_wp_measurement(sdss_sm10p0_gv_large, output_dirname, 'sdss_sm10p0_gv_large', 10., sm_high)
save_wp_measurement(sdss_sm10p0_q_small, output_dirname, 'sdss_sm10p0_q_small', 10., sm_high)
save_wp_measurement(sdss_sm10p0_q_large, output_dirname, 'sdss_sm10p0_q_large', 10., sm_high)


Measuring wp for sdss_sm10p0_sf_small sample of 22641 galaxies
Measuring wp for sdss_sm10p0_sf_large sample of 22220 galaxies
Measuring wp for sdss_sm10p0_gv_small sample of 9618 galaxies
Measuring wp for sdss_sm10p0_gv_large sample of 9304 galaxies
Measuring wp for sdss_sm10p0_q_small sample of 59746 galaxies
Measuring wp for sdss_sm10p0_q_large sample of 56930 galaxies


In [18]:
save_wp_measurement(sdss_sm10p5_sf_small, output_dirname, 'sdss_sm10p5_sf_small', 10.5, sm_high)
save_wp_measurement(sdss_sm10p5_sf_large, output_dirname, 'sdss_sm10p5_sf_large', 10.5, sm_high)
save_wp_measurement(sdss_sm10p5_gv_small, output_dirname, 'sdss_sm10p5_gv_small', 10.5, sm_high)
save_wp_measurement(sdss_sm10p5_gv_large, output_dirname, 'sdss_sm10p5_gv_large', 10.5, sm_high)
save_wp_measurement(sdss_sm10p5_q_small, output_dirname, 'sdss_sm10p5_q_small', 10.5, sm_high)
save_wp_measurement(sdss_sm10p5_q_large, output_dirname, 'sdss_sm10p5_q_large', 10.5, sm_high)



Measuring wp for sdss_sm10p5_sf_small sample of 12664 galaxies
Measuring wp for sdss_sm10p5_sf_large sample of 12573 galaxies
Measuring wp for sdss_sm10p5_gv_small sample of 7200 galaxies
Measuring wp for sdss_sm10p5_gv_large sample of 7027 galaxies
Measuring wp for sdss_sm10p5_q_small sample of 50376 galaxies
Measuring wp for sdss_sm10p5_q_large sample of 48391 galaxies


In [19]:
save_wp_measurement(sdss_sm10p0_meert15_disk_dom_small, output_dirname, 'sdss_sm10p0_meert15_disk_dom_small', 10., sm_high)
save_wp_measurement(sdss_sm10p0_meert15_disk_dom_large, output_dirname, 'sdss_sm10p0_meert15_disk_dom_large', 10., sm_high)
save_wp_measurement(sdss_sm10p0_meert15_bulge_dom_small, output_dirname, 'sdss_sm10p0_meert15_bulge_dom_small', 10., sm_high)
save_wp_measurement(sdss_sm10p0_meert15_bulge_dom_large, output_dirname, 'sdss_sm10p0_meert15_bulge_dom_large', 10., sm_high)




Measuring wp for sdss_sm10p0_meert15_disk_dom_small sample of 23054 galaxies
Measuring wp for sdss_sm10p0_meert15_disk_dom_large sample of 22639 galaxies
Measuring wp for sdss_sm10p0_meert15_bulge_dom_small sample of 14677 galaxies
Measuring wp for sdss_sm10p0_meert15_bulge_dom_large sample of 13870 galaxies


In [20]:
save_wp_measurement(sdss_sm10p5_meert15_disk_dom_small, output_dirname, 'sdss_sm10p5_meert15_disk_dom_small', 10.5, sm_high)
save_wp_measurement(sdss_sm10p5_meert15_disk_dom_large, output_dirname, 'sdss_sm10p5_meert15_disk_dom_large', 10.5, sm_high)
save_wp_measurement(sdss_sm10p5_meert15_bulge_dom_small, output_dirname, 'sdss_sm10p5_meert15_bulge_dom_small', 10.5, sm_high)
save_wp_measurement(sdss_sm10p5_meert15_bulge_dom_large, output_dirname, 'sdss_sm10p5_meert15_bulge_dom_large', 10.5, sm_high)


Measuring wp for sdss_sm10p5_meert15_disk_dom_small sample of 14412 galaxies
Measuring wp for sdss_sm10p5_meert15_disk_dom_large sample of 14176 galaxies
Measuring wp for sdss_sm10p5_meert15_bulge_dom_small sample of 12443 galaxies
Measuring wp for sdss_sm10p5_meert15_bulge_dom_large sample of 11906 galaxies


In [ ]:
fig, ax = plt.subplots(1, 1)

__=ax.loglog()

wp_dirname = "../../galsize_models/measurements/data/two_point_functions/"
wp_fname = os.path.join(wp_dirname, 'sdss_sm10p0_wp.npy')
wp2_fname = os.path.join(wp_dirname, 'sdss_sm10p5_wp.npy')
rp_fname = os.path.join(wp_dirname, 'sdss_sm10p0_rp.npy')
rp = np.load(rp_fname)
wp = np.load(wp_fname)
wp2 = np.load(wp2_fname)

__=ax.plot(rp, wp)
__=ax.plot(rp, wp2)

In [ ]:
from scipy.stats import binned_statistic

goodmask = ~np.isnan(umachine_sdss['r50_magr_kpc_meert15'])
x = umachine_sdss['sm'][goodmask]
values = umachine_sdss['r50_magr_kpc_meert15'][goodmask]
median_size_all, sm_bins_all, __  = binned_statistic(x, values, statistic='median')
sm_mids_all = 0.5*(sm_bins_all[:-1] + sm_bins_all[1:])
size_cut_all = np.interp(umachine_sdss['sm'], sm_mids_all, median_size_all)
low_size_mask_all = umachine_sdss['r50_magr_kpc_meert15'] < size_cut_all
high_size_mask_all = umachine_sdss['r50_magr_kpc_meert15'] >= size_cut_all

qmask = goodmask & (umachine_sdss['ssfr'] < -11)
x = umachine_sdss['sm'][qmask]
values = umachine_sdss['r50_magr_kpc_meert15'][qmask]
median_size_q, sm_bins_q, __ = binned_statistic(x, values, statistic='median')
sm_mids_q = 0.5*(sm_bins_q[:-1] + sm_bins_q[1:])
size_cut_q = np.interp(umachine_sdss['sm'], sm_mids_q, median_size_q)
low_size_mask_q = (umachine_sdss['r50_magr_kpc_meert15'] < size_cut_q)*(umachine_sdss['ssfr'] < -11)
high_size_mask_q = (umachine_sdss['r50_magr_kpc_meert15'] >= size_cut_q)*(umachine_sdss['ssfr'] < -11)

sfmask = goodmask & (umachine_sdss['ssfr'] < -11)
x = umachine_sdss['sm'][sfmask]
values = umachine_sdss['r50_magr_kpc_meert15'][sfmask]
median_size_sf, sm_bins_sf, __ = binned_statistic(x, values, statistic='median')
sm_mids_sf = 0.5*(sm_bins_sf[:-1] + sm_bins_sf[1:])
size_cut_sf = np.interp(umachine_sdss['sm'], sm_mids_sf, median_size_sf)
low_size_mask_sf = (umachine_sdss['r50_magr_kpc_meert15'] < size_cut_sf)*(umachine_sdss['ssfr'] >= -11)
high_size_mask_sf = (umachine_sdss['r50_magr_kpc_meert15'] >= size_cut_sf)*(umachine_sdss['ssfr'] >= -11)


In [ ]:
from galsize_models.measurements.measure_sdss_clustering import get_wp_measurements, write_umachine_ascii
import os
dirname = "/Users/aphearin/work/repositories/python/galsize_models/build/lib.macosx-10.9-x86_64-2.7"

q_fname = os.path.join(dirname, 'q.dat')
sf_fname = os.path.join(dirname, 'sf.dat')

low_sizes_q_fname = os.path.join(dirname, 'low_sizes_q.dat')
low_sizes_sf_fname = os.path.join(dirname, 'low_sizes_sf.dat')
high_sizes_q_fname = os.path.join(dirname, 'high_sizes_q.dat')
high_sizes_sf_fname = os.path.join(dirname, 'high_sizes_sf.dat')

low_sizes_all_fname = os.path.join(dirname, 'low_sizes_all.dat')
high_sizes_all_fname = os.path.join(dirname, 'high_sizes_all.dat')


write_umachine_ascii(umachine_sdss[low_size_mask_q], low_sizes_q_fname, overwrite=True)
write_umachine_ascii(umachine_sdss[high_size_mask_q], high_sizes_q_fname, overwrite=True)
write_umachine_ascii(umachine_sdss[low_size_mask_sf], low_sizes_sf_fname, overwrite=True)
write_umachine_ascii(umachine_sdss[high_size_mask_sf], high_sizes_sf_fname, overwrite=True)
write_umachine_ascii(umachine_sdss[high_size_mask_all], high_sizes_all_fname, overwrite=True)
write_umachine_ascii(umachine_sdss[low_size_mask_all], low_sizes_all_fname, overwrite=True)


write_umachine_ascii(umachine_sdss[qmask], q_fname, overwrite=True)
write_umachine_ascii(umachine_sdss[sfmask], sf_fname, overwrite=True)


In [ ]:
sm_low, sm_high, npts = 10, 12, 20

In [ ]:
low_size_q_results = get_wp_measurements(low_sizes_q_fname, sm_low, sm_high, npts)
rp_low_size_q, wp_low_size_q, wperr_low_size_q = low_size_q_results[:, 0], low_size_q_results[:, 1], low_size_q_results[:, 2]

low_size_sf_results = get_wp_measurements(low_sizes_sf_fname, sm_low, sm_high, npts)
rp_low_size_sf, wp_low_size_sf, wperr_low_size_sf = low_size_sf_results[:, 0], low_size_sf_results[:, 1], low_size_sf_results[:, 2]

In [ ]:
high_size_q_results = get_wp_measurements(high_sizes_q_fname, sm_low, sm_high, npts)
rp_high_size_q, wp_high_size_q, wperr_high_size_q = high_size_q_results[:, 0], high_size_q_results[:, 1], high_size_q_results[:, 2]

high_size_sf_results = get_wp_measurements(high_sizes_sf_fname, sm_low, sm_high, npts)
rp_high_size_sf, wp_high_size_sf, wperr_high_size_sf = high_size_sf_results[:, 0], high_size_sf_results[:, 1], high_size_sf_results[:, 2]

In [ ]:
q_results = get_wp_measurements(q_fname, sm_low, sm_high, npts)
rp_q, wp_q, wperr_q = q_results[:, 0], q_results[:, 1], q_results[:, 2]

sf_results = get_wp_measurements(sf_fname, sm_low, sm_high, npts)
rp_sf, wp_sf, wperr_sf = sf_results[:, 0], sf_results[:, 1], sf_results[:, 2]


In [ ]:
low_size_all_results = get_wp_measurements(low_sizes_all_fname, sm_low, sm_high, npts)
rp_low_size_all, wp_low_size_all, wperr_low_size_all = low_size_all_results[:, 0], low_size_all_results[:, 1], low_size_all_results[:, 2]

high_size_all_results = get_wp_measurements(high_sizes_all_fname, sm_low, sm_high, npts)
rp_high_size_all, wp_high_size_all, wperr_high_size_all = high_size_all_results[:, 0], high_size_all_results[:, 1], high_size_all_results[:, 2]


In [ ]:
np.save('sdss_fracdiff_sf_10p0', fracdiff_sf)


In [ ]:
np.save('sdss_rp_sf', rp_sf)

In [ ]:
wp_low_size_q_interp = np.interp(rp_q, rp_low_size_q, wp_low_size_q)
wp_low_size_sf_interp = np.interp(rp_sf, rp_low_size_sf, wp_low_size_sf)
wp_high_size_q_interp = np.interp(rp_q, rp_high_size_q, wp_high_size_q)
wp_high_size_sf_interp = np.interp(rp_sf, rp_high_size_sf, wp_high_size_sf)

fracdiff_q = (wp_high_size_q_interp - wp_low_size_q_interp)/wp_q
fracdiff_sf = (wp_high_size_sf_interp - wp_low_size_sf_interp)/wp_sf

# np.save('fracdiff_sf_9p75', fracdiff_sf)

fracdiff_sf_10p0 = np.load('sdss_fracdiff_sf_10p0.npy')
fracdiff_sf_10p5 = np.load('sdss_fracdiff_sf_10p5.npy')

fig, ax = plt.subplots(1, 1)

xscale = ax.set_xscale('log')

# __=ax.plot(rp_q, fracdiff_q, color='red')
__=ax.plot(rp_sf[5:], fracdiff_sf_10p5[5:], label=r'SDSS: $M_{\ast} > 10^{10.5}M_{\odot}$')
__=ax.plot(rp_sf[5:], fracdiff_sf_10p0[5:], label=r'SDSS: $M_{\ast} > 10^{10}M_{\odot}$')

__=ax.plot(np.logspace(-2, 2, 100), np.zeros(100), '--', color='k')

xlim = ax.set_xlim(0.2, 20)
ylim = ax.set_ylim(-0.3, 0.3)

xlabel = ax.set_xlabel(r'$R_{\rm p}$  [Mpc]')
ylabel = ax.set_ylabel(r'$(w_{\rm p}^{\rm large} - w_{\rm p}^{\rm small})/w_{\rm p}^{\rm all}$')
title = ax.set_title('size-dependence of main sequence clustering')
legend = ax.legend()

# fig.savefig('size_dependence_sf_clustering_SDSS.png', 
#             bbox_extra_artists=[xlabel, ylabel], bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(1, 1)

__=ax.loglog()

__=ax.plot(rp_low_size_q, wp_low_size_q, color='red')
__=ax.plot(rp_high_size_q, wp_high_size_q, '--', color='red')

__=ax.plot(rp_low_size_sf, wp_low_size_sf, color='blue')
__=ax.plot(rp_high_size_sf, wp_high_size_sf, '--', color='blue')

xlim = ax.set_xlim(0.1, 20)
